In [9]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr
import re

base_path = '/cs/labs/danix/nadav_al/AnimeInterp/experiments/outputs'

movies = ["aiw", "aot", "lk", "sat", "up"]
models = ["AnimeInterp", "LoraInterp", "LoraCNInterp"]
objectives = {"AnimeInterp": [""],
              "LoraInterp": ["latents", "result", "both"],
              "LoraCNInterp": ["latents", "both"]}

In [3]:
columns = ["Movie", "Model", "Diff_Objective", "LoRA_Rank", "Valid_Index", "PSNR", "IE"]
df = pd.DataFrame(columns=columns)

In [7]:
def isLora(model):
    return model == "LoraInterp" or model == "LoraCNInterp"

def extract_rank(string):
    pattern = r'rk(\d+)'
    match = re.search(pattern, string)
    if match:
        return match.group(1)
    else:
        return None

In [16]:
all_data = []
for movie_name in os.listdir(base_path):
    movie = movie_name.lower()
    if not movie in movies:
        continue
    movie_path = os.path.join(base_path, movie_name)
    for model_name in os.listdir(movie_path):
        if not model_name in models:
            continue
        model_path = os.path.join(movie_path, model_name)
        if isLora(model_name):
            for objective in os.listdir(model_path):
                objective_path = os.path.join(model_path, objective, "2300")
                for test_dets in os.listdir(objective_path):
                    rank = extract_rank(test_dets)
                    results = pd.read_csv(os.path.join(objective_path, test_dets, "losses.csv")).drop(["Unnamed: 0"], axis=1)
                    
                    # Add movie, model, objective, and rank information to the results
                    results['movie'] = movie
                    results['model'] = model_name
                    results['objective'] = objective
                    results['rank'] = rank
                    
                    # Append the results to the all_data list
                    all_data.append(results)
        else:  # AnimeInterp
            results = pd.read_csv(os.path.join(model_path, "losses.csv"))
            # Add movie, model, objective, and rank information to the results
            results['movie'] = movie
            results['model'] = model_name
            results['objective'] = None
            results['rank'] = None
            
            # Append the results to the all_data list
            all_data.append(results)
            

# Concatenate all the individual DataFrames into one
final_df = pd.concat(all_data, ignore_index=True)

# Reorder the columns to match your desired output
final_df = final_df[['movie', 'model', 'objective', 'rank', 'psnr', 'ie']]

# Display the first few rows of the final DataFrame
print(final_df.head())
    
    

FileNotFoundError: [Errno 2] No such file or directory: '/cs/labs/danix/nadav_al/AnimeInterp/experiments/outputs/AIW/LoraInterp/latents/2300/test10_ms_rk10_drpt5/losses.csv'